In [0]:
import numpy as np 

In [0]:
def normalize_anchors(anchors, ground_truths):
    
    """
    Args:
        groundtruths : Ndarray, 1D arrray, example) [cx, cy, w, h, 
                                                     cx1, cy1, w1, h1 ...]

        anchors : Ndarray, 1D arrray, example) [gt_cx, gt_cy, gt_w, gt_h, 
                                                gt_cx1, gt_cy1, gt_w1, gt_h1 
                                                ...]
    Return:
        norm_anchors: Ndarray, 3D array shape [N_anchor, N_gt, 4]
        
    Description:
        anchors 와 ground truths 을 통해 거리 계산을 한다. 
    """
    # 1d array (N*4) to 2d array (N, 4)
    anchors_2d = np.reshape(anchors, (-1,4))
    gt_2d = np.reshape(gt, (-1,4))

    
    # (N_anchor, 2) -> (1, N_anchor, 2)
    expand_anchors = np.expand_dims(anchors_2d , axis=1)
    
    # (N_gt, 2) -> (N_gt, 1, 2)
    expand_gt = np.expand_dims(gt_2d , axis=0)
    
    # Calculate delta
    delta_x = (expand_gt[:,:,0] - expand_anchors[:,:,0])/ expand_anchors[:,:,2]
    delta_y = (expand_gt[:,:,1] - expand_anchors[:,:,1])/ expand_anchors[:,:,3]
    delta_w = np.log(expand_gt[:,:,2]) - np.log(expand_anchors[:,:,2])
    delta_h = np.log(expand_gt[:,:,3]) - np.log(expand_anchors[:,:,3])
    
    # Caution # 
    # dtype=np.float32 이 구문을 제거하면 default 가 int 이되서 
    # float 값을 대입하면 모두 0이 된다. 주의하자.
    norm_anchors = np.ones_like(expand_anchors - expand_gt, dtype=np.float32)
    norm_anchors[:, : , 0] = delta_x
    norm_anchors[:, : , 1] = delta_y    
    norm_anchors[:, : , 2] = delta_w    
    norm_anchors[:, : , 3] = delta_h    
    return norm_anchors


def generate_trainable_anchors(normalize_anchors, positive_mask):
    """
    Args:
        normalize_anchors: 3D array, shape = [N_anchor, N_gt, 4]
        
        positve_mask: Ndarray, 2D array, 
            anchor 로 사용할 것은 *1*로  
            anchor 로 사용하지 않을 것은 *-1* 로 표기
            example: [[1 ,-1 ], <-anchor1
                      [-1 ,-1], <-anchor2
                      [-1 ,-1], <-anchor3
                      [1 , 1 ], <-anchor4
                      [-1 , 1]] <-anchor5                                        
                       gt1 gt2
           위 예제에서 사용할 anchor 는 (gt1, anchor1), (gt2, anchor4), (gt2, anchor5) 

    Description:
    
        학습시킬수 있는 anchors을 생성합니다. 
        입력된 normalize_anchors 는 Shape 을 [N_anchor, N_gt, 4] 가집니다. 
        해당 vector 에서 postive_mask 에 표시된(1로 표기된) 좌표의 
        anchor 만 가져옵니다. 
        
        해당 anchor 을 가져와 shape 가 [N_anchor , 4] 인 anchor 에 넣습니다. 
        
        # Caution! #        
        만약 가져올 anchor 가 없으면 (예제 anchor3) -1 -1 -1 -1로 채운다
        만약 가져올 anchor 가 많다면 가장 오른쪽에 있는 (gt2, anchor4) anchor 을 선택한다.
        
        
    """
    # 
    indices_2d = np.where(positive_mask == 1)
    indices_2d = np.stack(indices_2d, axis=0).tolist()

    # trainable axis=0 기준으로 어디에다가 추출한 x,y,w,h 을 넣어야 할지 가리키는 indices
    indices = indices_2d[0]

    # delta 에서 해당 좌표를 가져온다 
    dx = normalize_anchors[:,:,0][indices_2d]    
    dy = normalize_anchors[:,:,1][indices_2d]    
    dw = normalize_anchors[:,:,2][indices_2d]    
    dh = normalize_anchors[:,:,3][indices_2d]    

    # stack 한다. 
    d_xywh = np.stack([dx, dy, dw, dh], axis=-1)
    # 
    ret_anchor = np.ones([len(normalize_anchors), 4], dtype=np.float32)*-1
    ret_anchor[indices] = d_xywh
    return ret_anchor
    

In [0]:
# Test Normalization
anchors = [0,0,5,6, 
           5,5,10,11,
           0,0,7,6]
gt = [3, 3, 8, 8, 
      4, 4, 8, 8]

pos_mask =np.asarray([[-1,1],
          [-1,-1],
          [1,1]])

anchors = normalize_anchors(anchors, gt)
trainable_anchors = generate_trainable_anchors(anchors, pos_mask)



In [0]:
# Test Normalization
anchors = [0,0,5,6, 
           5,5,10,11,
           0,0,7,6]
gt = [3, 3, 8, 8, 
      4, 4, 8, 8]
           
anchors_2d = np.reshape(anchors, (-1,4))
gt_2d = np.reshape(gt, (-1,4))
           
expand_anchors = np.expand_dims(anchors_2d , axis=1)
expand_gt = np.expand_dims(gt_2d , axis=0)

norm_anchors = np.ones_like(expand_anchors - expand_gt, dtype=np.float32)*-1

delta_x = (expand_gt[:,:,0] - expand_anchors[:,:,0])/ expand_anchors[:,:,2]
delta_y = (expand_gt[:,:,1] - expand_anchors[:,:,1])/ expand_anchors[:,:,3]
delta_w = np.log(expand_gt[:,:,2]) - np.log(expand_anchors[:,:,2])
delta_h = np.log(expand_gt[:,:,3]) - np.log(expand_anchors[:,:,3])

assert(delta_x.shape == delta_y.shape == delta_w.shape == delta_h.shape )
print(delta_x)
print(delta_y)
print(delta_w)
print(delta_h)

In [0]:
norm_anchors[:, :, 0] = delta_x
norm_anchors[:, :, 1] = delta_y
norm_anchors[:, :, 2] = delta_w
norm_anchors[:, :, 3] = delta_h

pos_mask =np.asarray([[-1,1],
                      [-1,-1],
                      [-1,1]])
indices_2d = np.where(pos_mask == 1)
indices_2d = np.stack(indices_2d, axis=0).tolist()
print(indices_2d)

delta_x
print(delta_x[indices_2d])
print(delta_y[indices_2d])
print(delta_w[indices_2d])
print(delta_h[indices_2d])

In [0]:
norm_anchors[:, :, 0] = delta_x
norm_anchors[:, :, 1] = delta_y
norm_anchors[:, :, 2] = delta_w
norm_anchors[:, :, 3] = delta_h

pos_mask =np.asarray([[-1,1],
                      [-1,-1],
                      [1,1]])
indices_2d = np.where(pos_mask == 1)
indices_2d = np.stack(indices_2d, axis=0).tolist()


In [0]:

print(norm_anchors)
indices_2d = indices_2d
indices = indices_2d[0]
x = delta_x[indices_2d]
y = delta_y[indices_2d]
w = delta_w[indices_2d]
h = delta_h[indices_2d]
# print(x)
# print(y)
# print(w)
# print(h)
print(pos_mask)
xywh = np.stack([x, y, w, h], axis=-1)


print(xywh)
ret_anchor = np.ones([len(norm_anchors), 4])*-1
ret_anchor[indices] = xywh
ret_anchor